In [1]:
import numpy as np
import pandas as pd
import torch
from chronos import Chronos2Pipeline, ChronosBoltPipeline

print("NumPy:", np.__version__)
print("Pandas:", pd.__version__)
print("Torch:", torch.__version__)
print("Chronos OK")


NumPy: 2.4.2
Pandas: 3.0.0
Torch: 2.10.0+cpu
Chronos OK


# Instantaneous Traffic Generation and Probabilistic Forecasting

## 1. Motivation

The original Bouygues telecom dataset provides traffic measurements averaged over **coarse time intervals** (one week per observation).  
While such averages are sufficient to capture long-term traffic trends, they fail to represent the **short-term variability and burstiness** that are critical for network dimensioning and worst-case analysis.

In practice, telecom networks operate at much finer time resolutions (e.g., 5-minute intervals), where traffic demand exhibits rapid fluctuations due to user activity, mobility, and application behavior.  
As a result, forecasting only averaged traffic values is not sufficient to evaluate the ability of forecasting models to anticipate peak demand.

The objective of this notebook is therefore to:
- reconstruct **instantaneous traffic signals** from weekly averages using a principled data augmentation framework,
- and evaluate **probabilistic forecasting models** on this fine-grained traffic.

---

## 2. Assumption on Traffic Measurements

We assume that each weekly traffic value corresponds to an average over multiple **5-minute measurements**.

Formally, for a given antenna sector $s$ and week $w$, the observed traffic value $\bar{T}_s(w)$ can be written as:

$$
\bar{T}_s(w) = \frac{1}{N} \sum_{k=1}^{N} T_s(w, k)
$$

where:
- $T_s(w, k)$ denotes the instantaneous traffic at 5-minute slot $k$,
- $N = \frac{7 \times 24 \times 60}{5} = 2016$ is the number of 5-minute slots per week.

The instantaneous values $T_s(w, k)$ are not observed and must be generated.

---

## 3. Data Augmentation Framework

To generate realistic instantaneous traffic, we follow the **conceptual framework described in Section II-C and Figure 6(a)** of the reference paper.

The key idea is to reconstruct a fine-grained traffic signal that:
- preserves the **mean traffic level** observed in the dataset,
- exhibits **bursty ON/OFF behavior** typical of real telecom traffic,
- and introduces short-term variability on top of long-term trends.

Rather than explicitly modeling users and packet arrivals, we adopt a simplified stochastic approach that reproduces the essential statistical properties of real traffic.

---

## 4. Instantaneous Traffic Generation Model

For each sector and each week, instantaneous traffic is generated using the following steps:

### 4.1 Fine-Grained Time Axis

Each weekly observation is divided into $N = 2016$ pseudo 5-minute time slots, representing instantaneous traffic measurements.

---

### 4.2 Bursty Activity Modeling (ON/OFF Process)

Traffic activity is modeled using a binary ON/OFF process:

$$
B(k) \in \{0, 1\}
$$

where:
- $B(k) = 1$ indicates an active (burst) period,
- $B(k) = 0$ indicates an inactive or low-traffic period.

This process introduces realistic traffic intermittency and burstiness.

---

### 4.3 Raw Instantaneous Traffic Generation

During active periods, instantaneous traffic values are drawn from a random distribution:

$$
X(k) =
\begin{cases}
\epsilon_k, & \text{if } B(k) = 1 \\
0, & \text{if } B(k) = 0
\end{cases}
$$

where $\epsilon_k$ is a positive random variable capturing short-term traffic fluctuations.

This step introduces variability and peak values that are absent from averaged measurements.

---

### 4.4 Mean Preservation via Normalization

To ensure consistency with the original dataset, the generated instantaneous traffic is normalized so that its average matches the observed weekly value:

$$
T(k) = \frac{\bar{T}_s(w)}{\frac{1}{N} \sum_{k=1}^{N} X(k)} \cdot X(k)
$$

This guarantees that:
- the weekly average traffic remains unchanged,
- while instantaneous variability is explicitly introduced.

---

## 5. Resulting Traffic Characteristics

The resulting instantaneous traffic signal:
- follows the long-term trend of the original weekly data,
- exhibits realistic short-term fluctuations and bursts,
- produces higher peak values relevant for network dimensioning.

This behavior is consistent with the qualitative illustration shown in Figure 6(a) of the reference paper.

---

## 6. Forecasting Objective

The augmented dataset enables a new forecasting task:

> **Given historical instantaneous traffic measurements, predict future instantaneous traffic values.**

Both **large language models (Chronos-2)** and **trained LSTM baselines** are evaluated under this setting.

---

## 7. Evaluation Metrics

Two complementary metrics are used:

- **Root Mean Square Error (RMSE)**  
  Evaluates point forecast accuracy using the median prediction.

- **Pinball Loss (Quantile Loss)**  
  Evaluates probabilistic forecasting accuracy for high quantiles (e.g., $q = 0.9$), which are critical for worst-case network dimensioning.

---

## 8. Summary

By reconstructing fine-grained instantaneous traffic from coarse averages, this notebook enables a realistic evaluation of probabilistic forecasting models in a telecom context.  
This approach bridges the gap between averaged datasets and operational network requirements, where anticipating peak demand is essential.


In [2]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

np.random.seed(42)


#### Function to save longs 

In [55]:
import os, json
import pandas as pd
from datetime import datetime

RESULTS_CSV = "results_instantaneous_lstm_vs_chronos.csv"

def log_run(row: dict, path=RESULTS_CSV):
    row = dict(row)
    row["timestamp_run"] = datetime.now().isoformat(timespec="seconds")
    df_row = pd.DataFrame([row])
    if os.path.exists(path):
        df_row.to_csv(path, mode="a", header=False, index=False)
    else:
        df_row.to_csv(path, mode="w", header=True, index=False)

def load_runs(path=RESULTS_CSV):
    return pd.read_csv(path) if os.path.exists(path) else pd.DataFrame()


In [9]:
df = pd.read_csv(
    "data/histo_trafic.csv",
    sep=";",
    encoding="latin1"
)

df = df.loc[:, ~df.columns.str.contains("^Unnamed")]

df.head()


,secteur,site,tstamp,trafic_mbps
0,T70721A,T70721,lundi 18 juin 2018,0.000000
1,T70721B,T70721,lundi 18 juin 2018,1.788239
2,T70721C,T70721,lundi 18 juin 2018,0.000000
3,T70722A,T70722,lundi 18 juin 2018,4.437942
4,T70722B,T70722,lundi 18 juin 2018,10.885752


In [10]:
df["tstamp_clean"] = df["tstamp"].str.replace(
    r"^[a-zA-Zéûîôàç]+\s+", "", regex=True
)

month_map = {
    "janvier": "January",
    "février": "February",
    "mars": "March",
    "avril": "April",
    "mai": "May",
    "juin": "June",
    "juillet": "July",
    "août": "August",
    "septembre": "September",
    "octobre": "October",
    "novembre": "November",
    "décembre": "December"
}

for fr, en in month_map.items():
    df["tstamp_clean"] = df["tstamp_clean"].str.replace(fr, en, regex=False)

In [11]:
df["tstamp"] = pd.to_datetime(df["tstamp_clean"], format="%d %B %Y")
df = df.drop(columns="tstamp_clean")

In [12]:
df = df.sort_values(["secteur", "tstamp"])
df.head()

,secteur,site,tstamp,trafic_mbps
1575,T36870A,T36870,2018-11-12,0.263481
1658,T36870A,T36870,2018-11-19,0.066913
1741,T36870A,T36870,2018-11-26,0.062066
1824,T36870A,T36870,2018-12-03,0.084320
1907,T36870A,T36870,2018-12-10,0.047759


In [13]:
N_SLOTS = 2016          # 5-minute slots per week
P_ON = 0.25             # burst probability
NOISE_STD_RATIO = 0.5   # relative variability


In [14]:
def generate_instantaneous_traffic(weekly_mean,
                                    n_slots=N_SLOTS,
                                    p_on=P_ON,
                                    noise_std_ratio=NOISE_STD_RATIO):
    """
    Generate instantaneous traffic from a weekly average value.
    """
    # ON / OFF activity
    on_off = np.random.binomial(1, p_on, size=n_slots)

    # Raw traffic during ON periods
    noise_std = noise_std_ratio * weekly_mean
    raw = np.random.normal(
        loc=weekly_mean,
        scale=noise_std,
        size=n_slots
    )

    raw = np.clip(raw, a_min=0, a_max=None)

    X = on_off * raw

    # Mean preservation
    if X.mean() > 0:
        T = weekly_mean * X / X.mean()
    else:
        T = np.zeros_like(X)

    return T


In [15]:
instantaneous_rows = []

for _, row in df.iterrows():
    inst_signal = generate_instantaneous_traffic(row["trafic_mbps"])

    for k, value in enumerate(inst_signal):
        instantaneous_rows.append({
            "secteur": row["secteur"],
            "site": row["site"],
            "week": row["tstamp"],
            "slot": k,
            "traffic": value
        })

instantaneous_df = pd.DataFrame(instantaneous_rows)
instantaneous_df.head()


,secteur,site,week,slot,traffic
0,T36870A,T36870,2018-11-12,0,0.000000
1,T36870A,T36870,2018-11-12,1,0.891498
2,T36870A,T36870,2018-11-12,2,0.000000
3,T36870A,T36870,2018-11-12,3,0.000000
4,T36870A,T36870,2018-11-12,4,0.000000


**NOTE**
This process took around 14 minutes.

That’s normal because: 2016 slots × ~25k rows ≈ 50 million points


In [16]:
check = (
    instantaneous_df
    .groupby(["secteur", "week"])["traffic"]
    .mean()
    .reset_index()
    .merge(
        df[["secteur", "tstamp", "trafic_mbps"]],
        left_on=["secteur", "week"],
        right_on=["secteur", "tstamp"],
        how="left"
    )
)

check["error"] = check["traffic"] - check["trafic_mbps"]
check["error"].describe()


count    2.448600e+04
mean     3.529467e-17
std      3.547446e-15
min     -5.684342e-14
25%      0.000000e+00
50%      0.000000e+00
75%      0.000000e+00
max      2.842171e-14
Name: error, dtype: float64

we see that: 
  - Mean error ≈ 0 up to floating-point precision
  - Errors are on the order of 1e-14 → machine epsilon
  - 25/50/75% at 0 → expected with floating noise
  
The generated instantaneous traffic preserves the original weekly average up to numerical precision.

**Sanity check.**  
The average of the generated instantaneous traffic matches the original weekly traffic values up to numerical precision (mean error ≈ $10^{-15}$), confirming that the augmentation preserves long-term traffic statistics while introducing short-term variability.


In [21]:
instantaneous_df = instantaneous_df.copy()

# Each slot = 5 minutes
instantaneous_df["timestamp"] = (
    instantaneous_df["week"]
    + pd.to_timedelta(instantaneous_df["slot"] * 5, unit="min")
)

instantaneous_df = instantaneous_df.sort_values(
    ["secteur", "timestamp"]
).reset_index(drop=True)

instantaneous_df.head()


,secteur,site,week,slot,traffic,timestamp
0,T36870A,T36870,2018-11-12,0,0.000000,2018-11-12 00:00:00
1,T36870A,T36870,2018-11-12,1,0.891498,2018-11-12 00:05:00
2,T36870A,T36870,2018-11-12,2,0.000000,2018-11-12 00:10:00
3,T36870A,T36870,2018-11-12,3,0.000000,2018-11-12 00:15:00
4,T36870A,T36870,2018-11-12,4,0.000000,2018-11-12 00:20:00


we do NOT want to feed 50M points to Chronos.

In [24]:
# we keep only the last week per sector
week_max = instantaneous_df.groupby("secteur")["week"].transform("max")
last_week_df = instantaneous_df.loc[instantaneous_df["week"] == week_max].copy()


In [25]:
chronos_df = last_week_df.rename(columns={"secteur":"id", "traffic":"target"})[["id","timestamp","target"]]


chronos_df.head()


,id,timestamp,target
536256,T36870A,2024-01-01 00:00:00,0.000000
536257,T36870A,2024-01-01 00:05:00,0.000000
536258,T36870A,2024-01-01 00:10:00,50.060216
536259,T36870A,2024-01-01 00:15:00,0.000000
536260,T36870A,2024-01-01 00:20:00,0.000000


In [29]:
# mark last row per id
is_last = chronos_df.groupby("id")["timestamp"].transform("max") == chronos_df["timestamp"]

context_df = chronos_df.loc[~is_last].copy()
test_df    = chronos_df.loc[is_last].copy()

print(context_df.columns)
print(test_df.columns)



Index(['id', 'timestamp', 'target'], dtype='str')
Index(['id', 'timestamp', 'target'], dtype='str')


In [ ]:
from chronos import Chronos2Pipeline

pipeline = Chronos2Pipeline.from_pretrained(
    "amazon/chronos-2",
    device_map="auto"
)


config.json: 0.00B [00:00, ?B/s]

c:\Users\Sherif\miniconda3\Lib\site-packages\huggingface_hub\file_download.py:143: UserWarning: `huggingface_hub` cache-system uses symlinks by default to efficiently store duplicated files but your machine does not support them in C:\Users\Sherif\.cache\huggingface\hub\models--amazon--chronos-2. Caching files will still work but in a degraded version that might require more space on your disk. This warning can be disabled by setting the `HF_HUB_DISABLE_SYMLINKS_WARNING` environment variable. For more details, see https://huggingface.co/docs/huggingface_hub/how-to-cache#limitations.
To support symlinks on Windows, you either need to activate Developer Mode or to run Python as an administrator. In order to activate developer mode, see this article: https://docs.microsoft.com/en-us/windows/apps/get-started/enable-your-device-for-development
  warnings.warn(message)
Xet Storage is enabled for this repo, but the 'hf_xet' package is not installed. Falling back to regular HTTP download. For 

model.safetensors:   0%|          | 0.00/478M [00:00<?, ?B/s]

In [30]:
pred_df = pipeline.predict_df(
    context_df,
    prediction_length=1,
    id_column="id",
    timestamp_column="timestamp",
    target="target",
    quantile_levels=[0.5, 0.9],
)

pred_df.head()


c:\Users\Sherif\miniconda3\Lib\site-packages\chronos\chronos2\dataset.py:89: UserWarning: The given NumPy array is not writable, and PyTorch does not support non-writable tensors. This means writing to this tensor will result in undefined behavior. You may want to copy the array to protect its data or make it writable before converting it to a tensor. This type of warning will be suppressed for the rest of this program. (Triggered internally at C:\actions-runner\_work\pytorch\pytorch\pytorch\torch\csrc\utils\tensor_numpy.cpp:219.)
  task_target = torch.from_numpy(task_target)


,id,timestamp,target_name,predictions,0.5,0.9
0,T36870A,2024-01-07 23:55:00,target,0.243373,0.243373,73.116631
1,T36870B,2024-01-07 23:55:00,target,0.087078,0.087078,24.401529
2,T70721A,2024-01-07 23:55:00,target,0.124113,0.124113,19.745817
3,T70721B,2024-01-07 23:55:00,target,0.094389,0.094389,18.486391
4,T70721C,2024-01-07 23:55:00,target,0.026159,0.026159,6.536980


In [32]:
pred_df.head()
pred_df.dtypes


id                        str
timestamp      datetime64[us]
target_name               str
predictions           float32
0.5                   float32
0.9                   float32
dtype: object

In [40]:
eval_df = test_df.merge(
    pred_df[["id", "0.5", "0.9"]],
    on="id",
    how="inner"
)

eval_df = eval_df.rename(columns={"0.5": "pred_q50", "0.9": "pred_q90"})

eval_df.head()


,id,timestamp,target,pred_q50,pred_q90
0,T36870A,2024-01-07 23:55:00,97.373674,0.243373,73.116631
1,T36870B,2024-01-07 23:55:00,0.000000,0.087078,24.401529
2,T70721A,2024-01-07 23:55:00,0.000000,0.124113,19.745817
3,T70721B,2024-01-07 23:55:00,0.000000,0.094389,18.486391
4,T70721C,2024-01-07 23:55:00,0.000000,0.026159,6.536980


In [41]:
rmse_chronos = np.sqrt(
    np.mean((eval_df["pred_q50"] - eval_df["target"]) ** 2)
)

pinball_chronos_q90 = np.mean(
    np.maximum(
        0.9 * (eval_df["target"] - eval_df["pred_q90"]),
        0.1 * (eval_df["pred_q90"] - eval_df["target"])
    )
)

rmse_chronos, pinball_chronos_q90


(np.float64(112.9518842328321), np.float64(21.52474884773724))

The RMSE obtained on instantaneous traffic is significantly higher than on weekly averaged traffic.
This increase is expected, as instantaneous traffic exhibits high variance and bursty behavior that is absent from averaged measurements.
Consequently, point-wise metrics such as RMSE become less informative, motivating the use of probabilistic metrics such as the pinball loss for worst-case evaluation.

In [56]:
log_run({
    "model": "Chronos-2",
    "L": None,
    "hidden": None,
    "layers": None,
    "batch": None,
    "epochs": None,
    "lr": None,
    "rmse_q50": float(rmse_chronos),
    "pinball_q90": float(pinball_chronos_q90),
})


In [51]:
L = 128          # context length (5-min steps)
BATCH = 256
EPOCHS = 20     # start small
LR = 1e-3


we create per-sector sequences (one week each)

In [43]:
# we use last_week_df (contains secteur, timestamp, traffic)
week_series = (
    last_week_df
    .sort_values(["secteur", "timestamp"])
    .groupby("secteur")["traffic"]
    .apply(lambda s: s.to_numpy(dtype=np.float32))
)

# we keep only sectors with full week (or at least L+2 points)
week_series = week_series[week_series.apply(len) >= (L + 2)]
len(week_series)


86

We train windows + test point (last point)

For each sector:

 - train windows predict next-step within the week except last point

 - test uses the last L points → predict final point (same logic as Chronos)

In [44]:
X_train, y_train = [], []
X_test, y_test, test_ids = [], [], []

for sid, arr in week_series.items():
    n = len(arr)

    # training windows: use up to n-1 (leave last for final evaluation)
    train_end = n - 1
    for t in range(L, train_end):
        X_train.append(arr[t-L:t])
        y_train.append(arr[t])

    # test: last step prediction (t = n-1)
    X_test.append(arr[n-1-L:n-1])
    y_test.append(arr[n-1])
    test_ids.append(sid)

X_train = np.stack(X_train)
y_train = np.array(y_train, dtype=np.float32)

X_test  = np.stack(X_test)
y_test  = np.array(y_test, dtype=np.float32)

X_train.shape, X_test.shape


((162282, 128), (86, 128))

In [45]:
import torch
from torch import nn
from torch.utils.data import Dataset, DataLoader

class WindowDataset(Dataset):
    def __init__(self, X, y):
        self.X = torch.tensor(X, dtype=torch.float32).unsqueeze(-1)  # (N,L,1)
        self.y = torch.tensor(y, dtype=torch.float32).unsqueeze(-1)  # (N,1)
    def __len__(self): return len(self.X)
    def __getitem__(self, i): return self.X[i], self.y[i]

train_loader = DataLoader(WindowDataset(X_train, y_train), batch_size=BATCH, shuffle=True)


Two outputs: q50 and q90.

In [46]:
class QuantileLSTM(nn.Module):
    def __init__(self, hidden=64, layers=1):
        super().__init__()
        self.lstm = nn.LSTM(input_size=1, hidden_size=hidden, num_layers=layers, batch_first=True)
        self.head = nn.Linear(hidden, 2)  # two quantiles
    def forward(self, x):
        out, _ = self.lstm(x)          # (B,L,H)
        h = out[:, -1, :]              # last state
        q = self.head(h)               # (B,2)
        return q


In [47]:
def pinball(y, yhat, q):
    # y, yhat: (B,)
    e = y - yhat
    return torch.mean(torch.maximum(q*e, (1-q)*(-e)))

def quantile_loss(y, qhat):
    # qhat: (B,2) -> [q50, q90]
    y = y.squeeze(-1)
    q50 = qhat[:, 0]
    q90 = qhat[:, 1]
    return pinball(y, q50, 0.5) + pinball(y, q90, 0.9)


In [52]:
device = "cuda" if torch.cuda.is_available() else "cpu"

model = QuantileLSTM(hidden=64, layers=1).to(device)
opt = torch.optim.Adam(model.parameters(), lr=LR)

model.train()
for epoch in range(1, EPOCHS+1):
    losses = []
    for xb, yb in train_loader:
        xb, yb = xb.to(device), yb.to(device)
        opt.zero_grad()
        qhat = model(xb)
        loss = quantile_loss(yb, qhat)
        loss.backward()
        opt.step()
        losses.append(loss.item())
    print(f"Epoch {epoch}/{EPOCHS} - loss={np.mean(losses):.4f}")


Epoch 1/20 - loss=48.7488
Epoch 2/20 - loss=45.5131
Epoch 3/20 - loss=44.0318
Epoch 4/20 - loss=43.1740
Epoch 5/20 - loss=41.5104
Epoch 6/20 - loss=40.1466
Epoch 7/20 - loss=39.3231
Epoch 8/20 - loss=38.7755
Epoch 9/20 - loss=38.3743
Epoch 10/20 - loss=38.2931
Epoch 11/20 - loss=38.0483
Epoch 12/20 - loss=37.8299
Epoch 13/20 - loss=37.6920
Epoch 14/20 - loss=37.9135
Epoch 15/20 - loss=37.7607
Epoch 16/20 - loss=37.5951
Epoch 17/20 - loss=37.4862
Epoch 18/20 - loss=37.7594
Epoch 19/20 - loss=37.6235
Epoch 20/20 - loss=37.3418


In [53]:
model.eval()
with torch.no_grad():
    xb = torch.tensor(X_test, dtype=torch.float32).unsqueeze(-1).to(device)  # (S,L,1)
    qhat = model(xb).cpu().numpy()  # (S,2)

pred_q50 = qhat[:, 0]
pred_q90 = qhat[:, 1]

rmse_lstm = np.sqrt(np.mean((pred_q50 - y_test)**2))

pinball_lstm_q90 = np.mean(
    np.maximum(0.9*(y_test - pred_q90), 0.1*(pred_q90 - y_test))
)

rmse_lstm, pinball_lstm_q90


(np.float32(113.332886), np.float32(21.210234))

In [54]:
results = pd.DataFrame({
    "model": ["Chronos-2", "LSTM-quantile"],
    "RMSE_q50": [rmse_chronos, rmse_lstm],
    "Pinball_q90": [pinball_chronos_q90, pinball_lstm_q90]
})
results


,model,RMSE_q50,Pinball_q90
0,Chronos-2,112.951884,21.524749
1,LSTM-quantile,113.332886,21.210234


In [58]:
import os
import numpy as np
import pandas as pd
import torch
from torch import nn
from torch.utils.data import Dataset, DataLoader
from datetime import datetime

# -------------------------
# 0) Logging (CSV)
# -------------------------
RESULTS_CSV = "results_grid_lstm_quantiles.csv"

def log_run(row: dict, path=RESULTS_CSV):
    row = dict(row)
    row["run_time"] = datetime.now().isoformat(timespec="seconds")
    df_row = pd.DataFrame([row])
    if os.path.exists(path):
        df_row.to_csv(path, mode="a", header=False, index=False)
    else:
        df_row.to_csv(path, mode="w", header=True, index=False)

# -------------------------
# 1) Dataset helpers
# -------------------------
class WindowDataset(Dataset):
    def __init__(self, X, y):
        self.X = torch.tensor(X, dtype=torch.float32).unsqueeze(-1)  # (N,L,1)
        self.y = torch.tensor(y, dtype=torch.float32).unsqueeze(-1)  # (N,1)
    def __len__(self): return len(self.X)
    def __getitem__(self, i): return self.X[i], self.y[i]

def build_windows(week_series, L):
    """
    week_series: pd.Series mapping sector_id -> np.array traffic for last week
    Returns: X_train, y_train, X_test, y_test, test_ids
    """
    X_train, y_train = [], []
    X_test, y_test, test_ids = [], [], []

    for sid, arr in week_series.items():
        n = len(arr)
        if n < L + 2:
            continue

        # training: predict next step within the week, excluding final point
        train_end = n - 1
        for t in range(L, train_end):
            X_train.append(arr[t-L:t])
            y_train.append(arr[t])

        # test: predict last point
        X_test.append(arr[n-1-L:n-1])
        y_test.append(arr[n-1])
        test_ids.append(sid)

    X_train = np.stack(X_train).astype(np.float32)
    y_train = np.array(y_train, dtype=np.float32)

    X_test  = np.stack(X_test).astype(np.float32)
    y_test  = np.array(y_test, dtype=np.float32)

    return X_train, y_train, X_test, y_test, test_ids

# -------------------------
# 2) Model + loss
# -------------------------
class QuantileLSTM(nn.Module):
    def __init__(self, hidden=64, layers=1):
        super().__init__()
        self.lstm = nn.LSTM(input_size=1, hidden_size=hidden, num_layers=layers, batch_first=True)
        self.head = nn.Linear(hidden, 2)  # q50, q90
    def forward(self, x):
        out, _ = self.lstm(x)
        h = out[:, -1, :]
        return self.head(h)

def pinball_torch(y, yhat, q):
    e = y - yhat
    return torch.mean(torch.maximum(q*e, (1-q)*(-e)))

def quantile_loss(y, qhat):
    y = y.squeeze(-1)
    q50 = qhat[:, 0]
    q90 = qhat[:, 1]
    return pinball_torch(y, q50, 0.5) + pinball_torch(y, q90, 0.9)

def eval_metrics(y_true, pred_q50, pred_q90):
    rmse = float(np.sqrt(np.mean((pred_q50 - y_true) ** 2)))
    pinball_q90 = float(np.mean(np.maximum(0.9*(y_true - pred_q90), 0.1*(pred_q90 - y_true))))
    return rmse, pinball_q90

# -------------------------
# 3) One training run
# -------------------------
def train_and_eval_one_run(
    week_series,
    L=128,
    hidden=64,
    layers=1,
    epochs=5,
    batch=256,
    lr=1e-3,
    device=None
):
    if device is None:
        device = "cuda" if torch.cuda.is_available() else "cpu"

    X_train, y_train, X_test, y_test, test_ids = build_windows(week_series, L)

    train_loader = DataLoader(WindowDataset(X_train, y_train), batch_size=batch, shuffle=True)

    model = QuantileLSTM(hidden=hidden, layers=layers).to(device)
    opt = torch.optim.Adam(model.parameters(), lr=lr)

    # Train
    model.train()
    for epoch in range(1, epochs + 1):
        losses = []
        for xb, yb in train_loader:
            xb, yb = xb.to(device), yb.to(device)
            opt.zero_grad()
            qhat = model(xb)
            loss = quantile_loss(yb, qhat)
            loss.backward()
            opt.step()
            losses.append(loss.item())

    # Eval
    model.eval()
    with torch.no_grad():
        xb = torch.tensor(X_test, dtype=torch.float32).unsqueeze(-1).to(device)
        qhat = model(xb).cpu().numpy()

    pred_q50 = qhat[:, 0]
    pred_q90 = qhat[:, 1]
    rmse, pinball_q90 = eval_metrics(y_test, pred_q50, pred_q90)

    return rmse, pinball_q90, int(len(test_ids)), int(len(X_train))

# -------------------------
# 4) GRID LOOP
# -------------------------
device = "cuda" if torch.cuda.is_available() else "cpu"

grid_L      = [64, 128, 256]
grid_hidden = [32, 64, 128]
grid_layers = [1, 2]
grid_epochs = [3, 5, 10]
grid_lr     = [1e-3]
grid_batch  = [256]


# log_run({"model": "Chronos-2", "rmse_q50": float(rmse_chronos), "pinball_q90": float(pinball_chronos_q90)})

for L in grid_L:
    for hidden in grid_hidden:
        for layers in grid_layers:
            for epochs in grid_epochs:
                for lr in grid_lr:
                    for batch in grid_batch:
                        rmse_lstm, pinball_lstm_q90, n_sectors, n_train_windows = train_and_eval_one_run(
                            week_series=week_series,
                            L=L,
                            hidden=hidden,
                            layers=layers,
                            epochs=epochs,
                            batch=batch,
                            lr=lr,
                            device=device
                        )

                        log_run({
                            "model": "LSTM-quantile",
                            "L": L,
                            "hidden": hidden,
                            "layers": layers,
                            "epochs": epochs,
                            "batch": batch,
                            "lr": lr,
                            "n_sectors": n_sectors,
                            "n_train_windows": n_train_windows,
                            "rmse_q50": rmse_lstm,
                            "pinball_q90": pinball_lstm_q90
                        })

                        print(f"[DONE] L={L} hidden={hidden} layers={layers} epochs={epochs} "
                              f"-> RMSE={rmse_lstm:.3f} | Pinball@0.9={pinball_lstm_q90:.3f}")

print(f"\nSaved all runs to: {RESULTS_CSV}")


[DONE] L=64 hidden=32 layers=1 epochs=3 -> RMSE=113.333 | Pinball@0.9=36.848
[DONE] L=64 hidden=32 layers=1 epochs=5 -> RMSE=113.333 | Pinball@0.9=33.241
[DONE] L=64 hidden=32 layers=1 epochs=10 -> RMSE=113.331 | Pinball@0.9=23.834
[DONE] L=64 hidden=32 layers=2 epochs=3 -> RMSE=113.327 | Pinball@0.9=36.842
[DONE] L=64 hidden=32 layers=2 epochs=5 -> RMSE=113.333 | Pinball@0.9=33.337
[DONE] L=64 hidden=32 layers=2 epochs=10 -> RMSE=113.332 | Pinball@0.9=28.487
[DONE] L=64 hidden=64 layers=1 epochs=3 -> RMSE=113.330 | Pinball@0.9=31.958
[DONE] L=64 hidden=64 layers=1 epochs=5 -> RMSE=113.327 | Pinball@0.9=27.497
[DONE] L=64 hidden=64 layers=1 epochs=10 -> RMSE=113.332 | Pinball@0.9=21.578
[DONE] L=64 hidden=64 layers=2 epochs=3 -> RMSE=113.333 | Pinball@0.9=32.024
[DONE] L=64 hidden=64 layers=2 epochs=5 -> RMSE=113.335 | Pinball@0.9=28.882
[DONE] L=64 hidden=64 layers=2 epochs=10 -> RMSE=113.331 | Pinball@0.9=22.795
[DONE] L=64 hidden=128 layers=1 epochs=3 -> RMSE=113.276 | Pinball@0.9=2

In [59]:
runs = pd.read_csv(RESULTS_CSV)
runs.sort_values("pinball_q90").head(10)


,model,L,hidden,layers,epochs,batch,lr,n_sectors,n_train_windows,rmse_q50,pinball_q90,run_time
32,LSTM-quantile,128,128,1,10,256,0.001,86,162282,113.331879,19.492071,2026-02-09T01:03:09
50,LSTM-quantile,256,128,1,10,256,0.001,86,151274,113.326302,19.901014,2026-02-09T07:13:33
53,LSTM-quantile,256,128,2,10,256,0.001,86,151274,113.333725,20.695364,2026-02-09T10:37:08
35,LSTM-quantile,128,128,2,10,256,0.001,86,162282,113.333237,20.755035,2026-02-09T02:51:46
14,LSTM-quantile,64,128,1,10,256,0.001,86,167786,113.328850,20.960730,2026-02-08T21:17:35
31,LSTM-quantile,128,128,1,5,256,0.001,86,162282,113.328461,21.296179,2026-02-09T00:36:55
8,LSTM-quantile,64,64,1,10,256,0.001,86,167786,113.332077,21.578300,2026-02-08T20:25:04
17,LSTM-quantile,64,128,2,10,256,0.001,86,167786,113.329323,21.911261,2026-02-08T22:37:59
44,LSTM-quantile,256,64,1,10,256,0.001,86,151274,113.332382,22.275776,2026-02-09T04:24:49
13,LSTM-quantile,64,128,1,5,256,0.001,86,167786,113.331657,22.402582,2026-02-08T21:02:39


# AUGMENTATION section 6(a)/(b)

We followed Section II-C by generating instantaneous traffic via an Interrupted Poisson Process (IPP) with ON/OFF dynamics. Parameters 
𝜏
,
𝜁
τ,ζ control burst duration; 
𝜆
λ and 
𝐸
(
𝜓
)
E(ψ) are derived from the target mean and variance using the paper’s closed-form equations

In [60]:
T = 300        # 5 minutes in seconds
dt = 5.0       # generate one value every 5 seconds -> 60 points per row
tau, zeta = 0.1, 0.5
alpha = 0.5    # std = alpha * mean (you can tune)


In [66]:
import numpy as np

def ipp_window(mu, sigma2, T=300, dt=1.0, tau=0.1, zeta=0.5, seed=None):
    """
    Paper-style traffic generation for one averaging window of duration T seconds.
    mu     = E(Psi) target mean over the window
    sigma2 = Var(Psi) target variance over the window
    tau, zeta = OFF->ON and ON->OFF transition rates (1/sec)
    dt = time resolution for the generated instantaneous series (sec)
    Returns: instantaneous array of length int(T/dt)
    """
    rng = np.random.default_rng(seed)
    n = int(T / dt)

    if mu <= 0:
        return np.zeros(n, dtype=np.float32)

    # Eq (11): lambda (arrival rate in ON state)
    term = (sigma2 / (mu**2)) - (2 * zeta) / (tau * T * (tau + zeta))
    if term <= 0:
        # infeasible region (Fig 6c). Clamp so it still runs.
        term = 1e-6

    lam = (tau + zeta) / (tau * T) * term  # arrivals per second (ON state)

    # Eq (12): mean per-arrival demand E(psi)
    Epsi = (tau + zeta) * mu / (tau * lam * T)

    # Discrete-time Markov ON/OFF from continuous rates
    p_on_to_off  = 1 - np.exp(-zeta * dt)
    p_off_to_on  = 1 - np.exp(-tau  * dt)

    # initial state from steady-state
    p_on = tau / (tau + zeta)
    on = rng.random() < p_on

    x = np.zeros(n, dtype=np.float32)

    for t in range(n):
        # state transition
        if on:
            if rng.random() < p_on_to_off:
                on = False
        else:
            if rng.random() < p_off_to_on:
                on = True

        # arrivals + demand
        if on:
            U = rng.poisson(lam * dt)  # arrivals in this dt slice
            if U > 0:
                psi = rng.exponential(scale=Epsi, size=U)
                x[t] = psi.sum()
    m = x.mean()
    if m > 0:
        x = x * (mu / m)
    else:
        x[:] = 0.0
    
    return x


In [67]:
T = 300          # 5 minutes
dt = 5.0         # generate every 5 seconds (60 points per 5 minutes)
tau, zeta = 0.1, 0.5
alpha = 0.5      # controls burstiness via variance (you can tune)

def generate_instantaneous_from_avg(mu):
    sigma2 = (alpha * mu)**2
    return ipp_window(mu=mu, sigma2=sigma2, T=T, dt=dt, tau=tau, zeta=zeta)

# Example: one row
mu = float(df.loc[0, "trafic_mbps"])
inst = generate_instantaneous_from_avg(mu)
inst.mean(), mu


(np.float32(0.0), 0.0)

In [68]:
instantaneous_rows = []

for _, row in df.iterrows():
    mu = float(row["trafic_mbps"])
    sigma2 = (alpha * mu) ** 2

    inst = ipp_window(mu=mu, sigma2=sigma2, T=T, dt=dt, tau=tau, zeta=zeta)

    # timestamps inside this 5-min window
    base_ts = row["tstamp"]
    n = len(inst)

    for k in range(n):
        instantaneous_rows.append({
            "secteur": row["secteur"],
            "site": row["site"],
            "tstamp_avg": base_ts,                          # original 5-min average time
            "k": k,
            "timestamp": base_ts + pd.to_timedelta(k*dt, unit="s"),
            "traffic": float(inst[k])
        })

instantaneous_df = pd.DataFrame(instantaneous_rows)
instantaneous_df.head()


,secteur,site,tstamp_avg,k,timestamp,traffic
0,T36870A,T36870,2018-11-12,0,2018-11-12 00:00:00,0.0
1,T36870A,T36870,2018-11-12,1,2018-11-12 00:00:05,0.0
2,T36870A,T36870,2018-11-12,2,2018-11-12 00:00:10,0.0
3,T36870A,T36870,2018-11-12,3,2018-11-12 00:00:15,0.0
4,T36870A,T36870,2018-11-12,4,2018-11-12 00:00:20,0.0


In [69]:
check = (
    instantaneous_df.groupby(["secteur", "tstamp_avg"])["traffic"].mean()
    .reset_index()
    .merge(df[["secteur", "tstamp", "trafic_mbps"]],
           left_on=["secteur", "tstamp_avg"],
           right_on=["secteur", "tstamp"],
           how="left")
)

check["error"] = check["traffic"] - check["trafic_mbps"]
check["error"].describe()


count    2.448600e+04
mean    -1.632953e+01
std      2.131940e+01
min     -1.928720e+02
25%     -2.442114e+01
50%     -9.258292e+00
75%     -9.866886e-07
max      1.313854e-05
Name: error, dtype: float64